In [ ]:
#%% Imports
import os
import shutil
import nibabel as nib
import numpy as np
import random
import pickle
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy.stats as st
import nilearn
import pandas as pd

from scipy.stats.stats import pearsonr
from matplotlib.colors import colorConverter
from glob import glob
from scipy import ndimage
from nilearn.image import resample_to_img, resample_img
from nilearn.masking import compute_background_mask, compute_epi_mask
from nilearn.plotting import plot_roi, plot_epi, plot_img, plot_anat
from scipy.spatial.distance import directed_hausdorff
from nipype.algorithms.metrics import Distance
from sklearn.metrics import roc_curve, auc, f1_score, precision_recall_curve, average_precision_score
from scipy import interp
from itertools import chain
from scipy.ndimage.morphology import binary_dilation, binary_erosion, binary_closing, binary_opening
from skimage.morphology import cube, octahedron, ball

In [ ]:
# Set working directory
os.chdir('/home/uziel/DISS')
test_flag = 1

In [ ]:
def data_to_file(data, path):
    out = open(path, "w")
    for line in data:
        print >> out, line
    out.close()

In [ ]:
if test_flag:
    root = './data_processed/ISLES2017/testing'
else:
    root = './data_processed/ISLES2017/training'

channels = {}
# channels - sequences os.path.join('../../../../../../', x) needed for deepmedic
channels['Channels_ADC'] = sorted([os.path.join('../../../../../../', y)
                                   for x in os.walk(root)
                                   for y in glob(os.path.join(x[0], '*ADC*.nii.gz'))
                                  ])
channels['Channels_MTT'] = sorted([os.path.join('../../../../../../', y)
                                   for x in os.walk(root)
                                   for y in glob(os.path.join(x[0], '*MTT*.nii.gz'))
                                  ])
channels['Channels_rCBF'] = sorted([os.path.join('../../../../../../', y)
                                    for x in os.walk(root)
                                    for y in glob(os.path.join(x[0], '*rCBF*.nii.gz'))
                                  ])
channels['Channels_rCBV'] = sorted([os.path.join('../../../../../../', y)
                                    for x in os.walk(root)
                                    for y in glob(os.path.join(x[0], '*rCBV*.nii.gz'))
                                  ])
channels['Channels_Tmax'] = sorted([os.path.join('../../../../../../', y)
                                    for x in os.walk(root)
                                    for y in glob(os.path.join(x[0], '*Tmax*.nii.gz'))
                                  ])
channels['Channels_TTP'] = sorted([os.path.join('../../../../../../', y)
                                   for x in os.walk(root)
                                   for y in glob(os.path.join(x[0], '*TTP*.nii.gz'))
                                  ])
# labels
channels['GtLabels'] = sorted([os.path.join('../../../../../../', y)
                               for x in os.walk(root)
                               for y in glob(os.path.join(x[0], '*OT*.nii.gz'))
                              ])
# masks
channels['RoiMasks'] = sorted([os.path.join('../../../../../../', y)
                               for x in os.walk(root)
                               for y in glob(os.path.join(x[0], '*mask.nii.gz'))
                              ])

In [ ]:
#############################################
##### FILES FOR DM_VF (AUGMENTED DATA) ######
#############################################
if test_flag:
    data_set = 'test'
else:
    data_set = 'train'

s_path = './ischleseg/deepmedic/versions/DM_VF'
data_set_path = os.path.join(s_path, 'configFiles/' + data_set)

for name, files in channels.iteritems():
    # save train channel files
    data_to_file(files, os.path.join(data_set_path, data_set + name + '.cfg'))
    
if test_flag:
    names = ['SMIR.ischleseg_' + 'DM_VF' + '.' + os.path.basename(x).split('.')[-3]
             for x in channels['Channels_MTT']]
    data_to_file(names, os.path.join(data_set_path, 'testNamesOfPredictions.cfg'))

**MANAGE PREDICTED SEGMENTATIONS**

In [ ]:
root = '/media/uziel/Pulga2/DISS/milestones_6/DM_VF'
root_data = './data/ISLES2017/testing'


channels = {}
channels['Channels_MTT'] = sorted([y
                                   for x in os.walk(root_data)
                                   for y in glob(os.path.join(x[0], '*MTT*.nii'))
                                  ])

# List predictions
predictions = [x for x in sorted(os.listdir(os.path.join(root, 'output/predictions/testSession/predictions/'))) if 'Segm' in x]

# Define structure for closing operation
struct = ball(3)

In [ ]:
for pred in predictions:
    # Load prediction
    img = nib.load(os.path.join(os.path.join(root,
                                             'output/predictions/testSession/predictions/'),
                                pred))
    
    # Upsample prediction
    code = pred.split('.')[-3].split('_')[-2]
    original = [x for x in channels['Channels_MTT'] if code in x][0]
    original_img = nib.load(original)
    pred_img = resample_img(img,
                            original_img.affine,
                            original_img.shape,
                            interpolation='nearest')
    
    # Apply closing operation
    data = binary_closing(pred_img.get_data(), struct, iterations=2)
    
    # Save prediction
    nib.save(nib.Nifti1Image(data.astype(np.uint8), original_img.affine),
             os.path.join(root, 'output/predictions/') + 'SMIR.ishleseg_VF.' + code + '.nii')    
    